# AWS Kinesis – Overview and Use Cases

## What is AWS Kinesis?

Amazon Kinesis is a fully managed platform for real-time data ingestion, processing, and analysis at scale. It enables collection and processing of large streams of data in near real-time.

A shard is the base unit of capacity in Amazon Kinesis Data Streams. It defines how much data your stream can ingest and deliver.


**Each Shard Provides:**

**Write capacity:**

    - 1 MB/sec OR 1,000 records/sec

**Read capacity:**

    - 2 MB/sec for shared throughput
    - Up to 2 MB/sec per consumer for enhanced fan-out

## Core Components of AWS Kinesis

### 1. Kinesis Data Streams (KDS)
- Real-time, low-latency ingestion
- Scalable by shards
- Producers and consumers architecture

**Use Cases:**
- Clickstream tracking
- Fraud detection
- Application telemetry

### 2. Kinesis Data Firehose
- Serverless delivery to S3, Redshift, OpenSearch, Splunk
- Supports Lambda transformations
- No management needed

**Use Cases:**
- ETL pipelines
- Log persistence
- Analytics ingestion

### 3. Kinesis Data Analytics
- SQL and Apache Flink for real-time stream analysis
- Windowed aggregations, filtering, joins
- Integrates with Data Streams and Firehose

**Use Cases:**
- Real-time analytics
- Stream enrichment
- Trend monitoring

### 4. Kinesis Video Streams
- Ingests and stores video streams
- Integrates with SageMaker and ML services

**Use Cases:**
- Surveillance and security
- Live stream analysis
- Video-based ML inference


## Comparison of Components

| Feature            | Data Streams | Data Firehose | Data Analytics | Video Streams |
|--------------------|--------------|----------------|----------------|----------------|
| Purpose            | Ingest data  | Deliver data   | Analyze stream | Analyze video  |
| Transformations    | Custom code  | Lambda         | SQL / Flink    | Frame analysis |
| Storage            | Temporary    | S3, Redshift   | N/A            | Built-in       |
| Latency            | Low          | Medium         | Low            | Varies         |
| Complexity         | Medium       | Low            | Medium         | Medium         |




## Common Use Cases

| Domain             | Use Case Example                              |
|--------------------|-----------------------------------------------|
| Analytics          | Real-time dashboards, metrics ingestion       |
| Security           | Intrusion detection, log processing           |
| IoT                | Sensor data processing                        |
| Retail             | Real-time clickstream for personalization     |
| Media              | Live video stream processing                  |
| Finance            | Trade analysis, fraud detection               |

----



# Spark Structured Streaming

## What is Spark Structured Streaming?

**Structured Streaming** is a scalable and fault-tolerant stream processing engine built on the **Spark SQL engine**. It allows developers to write streaming jobs the same way they write batch jobs using **DataFrames** and **SQL**.

**Key Features:**
- Uses **DataFrame API** and **Spark SQL** for stream processing
- Supports **exactly-once** semantics
- Provides **high-level abstractions** (e.g., windowing, joins, aggregations)
- Integrates seamlessly with **batch pipelines**
- Supports **event time** processing with **watermarking**

Structured Streaming treats a streaming computation as a series of small batch computations, called **micro-batches**.



### Stream Reader

The **stream reader** defines the source of streaming data.

**Common Sources:**
- Kafka
- File systems (CSV, JSON, Parquet, etc.)
- Rate (for testing)
- Delta tables

**Example:**

```python
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "topic_name") \
    .load()
````


### Stream Writer

The **stream writer** defines how the output should be written.

**Common Sink Formats:**

* console
* file (csv, json, parquet, delta)
* memory
* kafka
* table

**Example:**

```python
query = df.writeStream \
    .format("console") \
    .outputMode("append") \
    .start()
```

**Output Modes:**

* `append`: Only new rows
* `update`: Updated rows only
* `complete`: Full result table (used with aggregations)


### Checkpointing

**Checkpointing** stores metadata and intermediate state to allow:

* Fault tolerance
* Stateful processing
* Resuming streams on failure

**Usage:**

```python
.writeStream \
    .option("checkpointLocation", "/path/to/checkpoint") \
    .start()
```

Without checkpointing, stateful operations (like aggregations, joins) will fail.


## Realtime data processing patterns

### Windowing

**Windowing** enables time-based groupings for streaming data.

**Types of Windows:**

* **Tumbling Window**: Fixed-length, non-overlapping
* **Sliding Window**: Overlapping windows
* **Session Window**: Grouped by inactivity gaps

**Example:**

```python
from pyspark.sql.functions import window

df.groupBy(window("timestamp", "10 minutes")) \
  .count()
```


### Watermarking

**Watermarking** handles **late-arriving** data in event-time based processing.

**Why It's Important:**

* Allows system to clean up state
* Defines maximum allowed delay for late events

**Usage:**

```python
df.withWatermark("event_time", "10 minutes")
```

Used together with aggregations:

```python
df.withWatermark("event_time", "10 minutes") \
  .groupBy(window("event_time", "5 minutes")) \
  .count()
```


### State Management

**Stateful operations** need to track data across micro-batches.

**Examples of Stateful Operations:**

* Grouped aggregations
* Stream-stream joins
* Deduplication
* Custom state logic via `mapGroupsWithState`

**State Store:** Internally stores key-value state, periodically cleaned up using watermarking or timeouts.

**Example:**

```python
df.groupBy("id").count()  # Maintains state per ID
```


### Stream Processing
- **Definition**: Processes each incoming event individually in real-time.
- **Tools**: Apache Kafka, Apache Flink, Spark Structured Streaming, AWS Kinesis.
- **Use Case**: Fraud detection, stock price tracking.


### Micro-Batching
- **Definition**: Buffers data for a short duration (e.g., 1–2 seconds) and processes in mini-batches.
- **Tools**: Spark Structured Streaming (uses micro-batching internally).
- **Use Case**: Real-time log analysis with slight delay tolerance.


### Event-Driven Pattern (Lambda Architecture)
- **Definition**: Combines batch and stream processing.
  - **Speed Layer**: For real-time data.
  - **Batch Layer**: For historical data.
  - **Serving Layer**: Combines both for querying.
- **Use Case**: Real-time + historical analytics dashboards.


### Event Sourcing
- **Definition**: Records all changes as a sequence of immutable events instead of just the final state.
- **Use Case**: Financial systems with full audit trails, systems needing replay capability.

### Change Data Capture (CDC)
- **Definition**: Captures and streams changes in a database (insert, update, delete).
- **Tools**: Debezium, AWS DMS, Oracle GoldenGate.
- **Use Case**: Syncing OLTP DB to real-time warehouse.




## Summary Table

| Concept          | Description                                   | Use Case                           |
| ---------------- | --------------------------------------------- | ---------------------------------- |
| Stream Reader    | Reads real-time data from source              | Kafka, files, rate source          |
| Stream Writer    | Writes results to sink                        | Console, file, delta, memory       |
| Checkpointing    | Stores state and progress                     | Required for stateful ops          |
| Windowing        | Group data by event time intervals            | Time-based analytics               |
| Watermarking     | Handles late data and controls state cleanup  | Event-time aggregations            |
| State Management | Maintains intermediate results across batches | Aggregations, joins, deduplication |


## Best Practices for Structured Streaming

1. **Always Set a Checkpoint Location**

   * Required for fault-tolerance and stateful operations.

2. **Use Watermarking with Event Time**

   * To prevent unbounded state growth and handle late data properly.

3. **Optimize Trigger Intervals**

   * Use `trigger(processingTime="10 seconds")` to balance latency and throughput.

4. **Choose Output Mode Carefully**

   * Use `append` for insert-only data.
   * Use `update` or `complete` when dealing with aggregations.

5. **Use Efficient File Formats**

   * Prefer `parquet` or `delta` over `csv/json` for performance.

6. **Avoid Overloading Console Sink**

   * Console sink is for debugging only, not production.

7. **Monitor Streaming Jobs**

    * Use Spark UI and logs to track progress, throughput, and state size.


```
[IoT Devices / App Logs / APIs]
        ↓
   [Kafka / Kinesis]
        ↓
[Stream Processor (Flink / Spark / Beam)]
        ↓
[Realtime DB (Redis / Elastic) + Datalake (S3)]
        ↓
[Dashboards / Alerts / ML Models]
```



----

#### kinesis-connector for spark streaming

```sh
sudo wget https://awslabs-code-us-east-1.s3.amazonaws.com/spark-sql-kinesis-connector/spark-streaming-sql-kinesis-connector_2.12-1.2.1.jar
sudo chmod 755 spark-streaming-sql-kinesis-connector_2.12-1.2.1.jar
```

In [1]:
! pip install pyspark==3.5.0


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("KinesisConsumer") \
            .config("spark.jars", "/workspaces/trng-2286/spark-streaming-sql-kinesis-connector_2.12-1.2.1.jar")\
            .getOrCreate()


25/08/08 04:26:07 WARN Utils: Your hostname, codespaces-c6070e resolves to a loopback address: 127.0.0.1; using 10.0.0.88 instead (on interface eth0)
25/08/08 04:26:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/08/08 04:26:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
! pip install python-dotenv boto3


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

[aws kinesis docs](https://aws.amazon.com/blogs/big-data/build-spark-structured-streaming-applications-with-the-open-source-connector-for-amazon-kinesis-data-streams/)

In [3]:
kinesis = spark.readStream.format("aws-kinesis") \
    .option("kinesis.region", "us-east-1") \
    .option("kinesis.streamName", "demo-spark-streaming-events") \
    .option("kinesis.consumerType", "GetRecords") \
    .option("kinesis.endpointUrl", "https://kinesis.us-east-1.amazonaws.com") \
    .option("kinesis.startingposition", "LATEST") \
    .option("kinesis.awsAccessKeyId", os.getenv("AWS_ACCESS_KEY")) \
    .option("kinesis.awsSecretKey", os.getenv("AWS_SECRET_ACCESS_KEY")) \
    .load()

In [4]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

schema = StructType([
    StructField("id", IntegerType()),
    StructField("event", StringType()),
    StructField("value", DoubleType()),
    StructField("event_time", TimestampType())
])

parsed_df = kinesis \
    .selectExpr("CAST(data as STRING) as json_data") \
    .select(from_json("json_data", schema).alias("data")) \
    .select("data.*")



In [ ]:
aggrgated_df = parsed_df \
        .withWatermark("event_time", "15 minutes") \
        .groupBy(
            window("event_time", "10 minutes", "5 minutes"), # sliding window
            col("id")
        )


In [6]:
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", True)

In [8]:
query = parsed_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", "False") \
    .option("checkpointLocation", "kinesis-checkpoint") \
    .trigger(processingTime = "10 seconds") \
    .start()

query.awaitTermination()

25/08/08 04:27:04 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 4
-------------------------------------------
+---+--------+-----+--------------------------+
|id |event   |value|event_time                |
+---+--------+-----+--------------------------+
|359|purchase|43.01|2025-08-08 04:24:48.835473|
|953|purchase|95.12|2025-08-08 04:24:50.766439|
|67 |click   |10.93|2025-08-08 04:24:51.981412|
|667|view    |80.1 |2025-08-08 04:24:53.197143|
|265|purchase|70.77|2025-08-08 04:24:54.410997|
|363|purchase|51.15|2025-08-08 04:24:55.622517|
|840|view    |45.38|2025-08-08 04:24:56.836915|
|418|click   |95.0 |2025-08-08 04:24:58.052577|
|472|purchase|33.88|2025-08-08 04:24:59.26775 |
|702|view    |92.42|2025-08-08 04:25:00.480209|
|988|click   |49.43|2025-08-08 04:25:01.693096|
|269|click   |62.59|2025-08-08 04:25:02.907686|
+---+--------+-----+--------------------------+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/python/3.12.1/lib/python3.12/socket.py", line 707, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
query = parsed_df.writeStream \
    .outputMode("append") \
    .format("csv") \
    .option("path", "./datasets/events/") \
    .option("checkpointLocation", "kinesis-checkpoint") \
    .trigger(processingTime = "10 seconds") \
    .start()

query.awaitTermination()